In [21]:
import polars as pl
from deltalake import write_deltalake, DeltaTable
import os
from datetime import date
from pathlib import Path as path

In [39]:
bronze_path = path('C:/Users/dougl/OneDrive/Área de Trabalho/ESTUDOS/nubank-data-engineering-portfolio/data/bronze/credito')

In [ ]:
df_credito_delta = DeltaTable(bronze_path)

df_credito_pl = pl.from_arrow(df_credito_delta.to_pyarrow_table())

In [44]:
df_credito_renomeia_colunas =  df_credito_pl.rename({'date':'data', 'title':'descricao', 'amount':'valor'})

In [62]:
df_credito_tipagem = df_credito_renomeia_colunas.with_columns([
    pl.col('data').str.strptime(pl.Date)
])

display(df_credito_tipagem.head(5))

data,descricao,valor
date,str,f64
2023-01-01,"""iFood*Rest""",49.03
2023-01-03,"""AliExpress""",87.72
2023-01-03,"""Mercado Livre""",133.27
2023-01-04,"""Apple.com/bill""",168.72
2023-01-05,"""Drogasil""",189.6


In [70]:
df_credito_adiciona_parcela = df_credito_tipagem.with_columns([
    pl.when(pl.col('descricao').str.contains(r'(\d+/\d+)'))
    .then(pl.col('descricao').str.extract(r'(\d+/\d+)'))
    .otherwise(None)
    .alias('parcela')
])

df_filtrado = df_credito_adiciona_parcela.filter(pl.col("descricao").str.contains("Amazon"))
display(df_filtrado)

data,descricao,valor,parcela
date,str,f64,str
2023-01-28,"""Amazon Mktp""",151.84,null
2023-02-28,"""Amazon Mktp""",78.9,null
2024-02-22,"""Amazon Mktp""",98.97,null
2023-03-08,"""Amazon Mktp""",28.91,null
2024-04-04,"""Amazon Mktp""",17.69,null
…,…,…,…
2023-09-07,"""Amazon Mktp""",164.69,null
2023-09-12,"""Amazon Mktp 1/4""",419.06,"""1/4"""
2023-10-11,"""Amazon Mktp""",112.46,null
